# Object Detection Demo

This notebook uses a Convolutional Neural Network (CNN) model called [YOLO](https://pjreddie.com/darknet/yolo/) to detect objects in images. The CNN in this notebook is _pretrained_, meaning that the weights have been learned and saved to a file for us to use, without needed to train it from scratch.

Run the following cell to install the needed packages. You only need to run it once.

In [ ]:
using Pkg
Pkg.add("ObjectDetector")
Pkg.add("FileIO")
Pkg.add("ImageMagick")
Pkg.add("ImageDraw")
Pkg.add("ImageView")
Pkg.add("Gtk")

This cell imports needed libraries. If it doesn't work, try restarting the kernel. 

In [ ]:
using ObjectDetector
using FileIO
using Images
using ImageDraw
using ImageView
using Gtk.ShortNames

This cell loads the YOLO model and the class names from a file.

In [ ]:
# Load class names for YOLO.
class_names = Vector{String}()
open("coco_classes.txt") do f
    while !eof(f)        
        s = readline(f)
        push!(class_names, s)
    end
end

# Load the YOLOv3-tiny model pretrained on COCO, with a batch size of 1
yolomod = YOLO.v3_608_COCO(batch=1, silent=true)

In [ ]:
function drawBBoxes(img::AbstractArray, model::YOLO.yolo, padding::AbstractArray, results; transpose=true)
    imgratio = size(img,2) / size(img,1)
    if transpose
        modelratio = model.cfg[:width] / model.cfg[:height]
        x1i, y1i, x2i, y2i = [1, 2, 3, 4]
    else
        modelratio = model.cfg[:height] / model.cfg[:width]
        x1i, y1i, x2i, y2i = [2, 1, 4, 3]
    end
    if modelratio > imgratio
        h, w = size(img,1) .* (1, modelratio)
    else
        h, w = size(img,2) ./ (modelratio, 1)
    end
    
    colorArr = zeros(size(results,2),3)

    for i in 1:size(results,2)
        bbox = results[1:4, i] .- padding
        class = convert(Int, results[end-1, i]) + 1
        conf = results[5,i]
        topw, toph = round(Int, bbox[x1i]*w)+1, round(Int, bbox[y1i]*h)+1
        p = Point(topw, toph)
        q = Point(round(Int, bbox[x2i]*w), round(Int, bbox[y1i]*h)+1)
        r = Point(round(Int, bbox[x1i]*w)+1, round(Int, bbox[y2i]*h))
        s = Point(round(Int, bbox[x2i]*w), round(Int, bbox[y2i]*h))
        pol = Polygon([p,q,s,r])
        colorArr[i,:] = [rand(), rand(), rand()]
        draw!(img, pol, eltype(img)(colorArr[i,1],colorArr[i,2],colorArr[i,3]))
    end

    guidict = imshow(img)
    
    for i in 1:size(results,2)
        bbox = results[1:4, i] .- padding
        class = convert(Int, results[end-1, i]) + 1
        conf = results[5,i]
        topw, toph = round(Int, bbox[x1i]*w)+1, round(Int, bbox[y1i]*h)+1
        
        annotate!(guidict, AnnotationText(topw, toph-20, class_names[class], fontsize=20, color=RGB(colorArr[i,1], colorArr[i,2], colorArr[i,3])))
    end
    
    c = Condition()
    win = guidict["gui"]["window"]
    signal_connect(win, :destroy) do widget
        notify(c)
    end
    wait(c)
end

## Run the Object Detector

Run this cell to use the model to detect objects. It will look at all the objects in the folder `input`.

In [ ]:
input_imgs = readdir("input")
for img_fname in input_imgs
    println("Testing image: $img_fname")
    batch = emptybatch(yolomod) # Create a batch object. Automatically uses the GPU if available
    img = load(joinpath("input", img_fname))
    batch[:,:,:,1], padding = prepareImage(img, yolomod) # Send resized image to the batch
    res = yolomod(batch, detectThresh=0.5, overlapThresh=0.6) # Run the model on the length-1 batch

    labels = res[end-1,:] # See: https://github.com/r3tex/ObjectDetector.jl/blob/master/src/utils.jl#L53
    unique_labels = [convert(Int32, lbl) for lbl in unique(labels)] .+ 1

    @show unique_labels
    print("    labels: ")
    for num in unique_labels
        print(class_names[num], " ")
    end
    println()
    
    drawBBoxes(img, yolomod, padding, res)
end
println("Done processing images!")